## Genome Generation
Given a number of chromosome pairs and genes, randomly generates chromosomes with genes distributed at random indices throughout.

In [1]:

import random
import string
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import itertools
import numpy as np
import networkx as nx
from IPython.display import display

def genomecreate(chromosomes=3, total_genes=20, seed=407):
    random.seed(seed)
    
    def distribute_genes(total_genes, chromosomes, min_genes=6):
        while True:
            sizes = [min_genes] * chromosomes
            remaining = total_genes - sum(sizes)
            for _ in range(remaining):
                sizes[random.randint(0, chromosomes - 1)] += 1
            if sum(sizes) == total_genes:
                return sizes

    gene_counts = distribute_genes(total_genes, chromosomes)

    all_genes = list(string.ascii_uppercase[:total_genes])
    random.shuffle(all_genes)

    chromosomes_data = []
    gene_idx = 0

    for chr_num, gene_count in enumerate(gene_counts, 1):
        chr_length = random.randint(1000, 2000)
        gene_names = all_genes[gene_idx:gene_idx + gene_count]

        # Ensure no genes near chromosome tips
        min_pos = int(chr_length * 0.02)
        max_pos = int(chr_length * 0.98)

        # Ensure even distribution across the chromosome body
        quarter = (max_pos - min_pos) // 4
        required_positions = [
            random.randint(min_pos + i * quarter + 1, min_pos + (i + 1) * quarter)
            for i in range(4)
        ]

        remaining = gene_count - 4
        position_pool = list(range(min_pos, max_pos + 1))
        remaining_positions = random.sample(position_pool, remaining)

        unique_remaining = list(set(remaining_positions) - set(required_positions))
        while len(unique_remaining) < remaining:
            new_pos = random.randint(1, chr_length)
            if new_pos not in required_positions and new_pos not in unique_remaining:
                unique_remaining.append(new_pos)

        positions = sorted(required_positions + unique_remaining[:remaining])
        genes = list(zip(gene_names, positions))

        chrA = [(g, pos) for g, pos in genes]
        chrB = [(g, pos) for g, pos in genes]

        df = pd.DataFrame({
            "Chromosome": f"Chr{chr_num}",
            "Gene": gene_names,
            "Position": positions,
            "ChrA_Before": [g.upper() for g, _ in chrA],
            "ChrB_Before": [g.lower() for g, _ in chrB],
        })

        chromosomes_data.append((f"Chr{chr_num}", df, chr_length))
        gene_idx += gene_count

    return chromosomes_data

In [2]:
def print_genome(genome_data):
    for chr_name, df, chr_length in genome_data:
        print(f"\n=== {chr_name} | Length: {chr_length} bp ===")
        print(df[["Gene", "Position", "ChrA_Before", "ChrB_Before"]].to_string(index=False))

In [5]:
genome=genomecreate(3, 20, 407)
print_genome(genome)


=== Chr1 | Length: 1220 bp ===
Gene  Position ChrA_Before ChrB_Before
   C       234           C           c
   P       347           P           p
   S       485           S           s
   D       624           D           d
   H       715           H           h
   E       998           E           e
   F      1189           F           f

=== Chr2 | Length: 1269 bp ===
Gene  Position ChrA_Before ChrB_Before
   A        99           A           a
   T       516           T           t
   Q       891           Q           q
   B       975           B           b
   G       991           G           g
   N      1228           N           n

=== Chr3 | Length: 1190 bp ===
Gene  Position ChrA_Before ChrB_Before
   I       272           I           i
   K       347           K           k
   O       355           O           o
   L       440           L           l
   M       510           M           m
   R       819           R           r
   J       918           J           j


## Meiosis Simulator
Given a genome, simulates recombination, selection of chromosomes. Returns alleles found on resulting gamete.

In [17]:
def meiosis(chromosomes_data, seed=5403):
    if seed is not None:
        random.seed(seed)

    recombinant_data = []

    for chr_name, df, chr_length in chromosomes_data:
        # 50% chance that recombination occurs on this chromosome
        do_recombine = random.random() < 0.5

        if do_recombine:
            crossover = random.randint(1, chr_length)
            swap_above = random.random() < 0.5

            chr1 = list(zip(df["ChrA_Before"], df["Position"]))
            chr2 = list(zip(df["ChrB_Before"], df["Position"]))

            recombinant1 = []
            recombinant2 = []

            for (g1, pos), (g2, _) in zip(chr1, chr2):
                if (swap_above and pos > crossover) or (not swap_above and pos < crossover):
                    recombinant1.append((g2.lower(), pos))
                    recombinant2.append((g1.upper(), pos))
                else:
                    recombinant1.append((g1.upper(), pos))
                    recombinant2.append((g2.lower(), pos))

            df["ChrA_After"] = [g for g, _ in recombinant1]
            df["ChrB_After"] = [g for g, _ in recombinant2]

        else:
            # No recombination: "after" is just a copy of "before"
            df["ChrA_After"] = df["ChrA_Before"]
            df["ChrB_After"] = df["ChrB_Before"]

        recombinant_data.append((chr_name, None, None, df, chr_length))

    # Build gamete genotype
    gamete_alleles = []
    for chr_name, _, _, df, _ in recombinant_data:
        # Randomly choose one homolog (ChrA or ChrB) for the gamete
        selected = df["ChrA_After"] if random.random() < 0.5 else df["ChrB_After"]
        sorted_alleles = [allele for _, allele in sorted(zip(df["Position"], selected))]
        gamete_alleles.extend(sorted_alleles)

    return ",".join(sorted(gamete_alleles, key=str.lower))


## Reproduction
Given two gametes, returns the resulting offspring genotype

In [9]:
def fertilization(gamete1, gamete2):
    alleles1 = gamete1.split(",")
    alleles2 = gamete2.split(",")

    # Combine both lists and group by base lette
    allele_dict = {}

    for a in alleles1 + alleles2:
        base = a.lower()
        if base not in allele_dict:
            allele_dict[base] = []
        allele_dict[base].append(a)

    # Sort each genotype pair with capital allele first (if present)
    child_genotype = []
    for base in sorted(allele_dict.keys()):
        pair = allele_dict[base]
        if len(pair) == 1:
            pair.append(pair[0])
        sorted_pair = sorted(pair, key=lambda x: (x.islower(), x))
        child_genotype.append("".join(sorted_pair))

    return "/".join(child_genotype)

In [19]:
genotypes=[]

for _ in range(2000):
    g1 = meiosis(genome)
    genotype = fertilization(g1,"a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t")
    genotypes.append(genotype)
    df_genotypes = pd.DataFrame(genotypes, columns=["Genotype"])

print(df_genotypes.head())
#df_genotypes.to_csv("genotype_samples.csv", index=False)

                                            Genotype
0  aa/bb/Cc/Dd/Ee/Ff/gg/Hh/ii/jj/kk/ll/mm/nn/oo/P...
1  aa/bb/Cc/Dd/Ee/Ff/gg/Hh/ii/jj/kk/ll/mm/nn/oo/P...
2  aa/bb/Cc/Dd/Ee/Ff/gg/Hh/ii/jj/kk/ll/mm/nn/oo/P...
3  aa/bb/Cc/Dd/Ee/Ff/gg/Hh/ii/jj/kk/ll/mm/nn/oo/P...
4  aa/bb/Cc/Dd/Ee/Ff/gg/Hh/ii/jj/kk/ll/mm/nn/oo/P...


In [ ]:
genotypes2=[]

for _ in range(1000):
    g1 = meiosis(genome)
    g2=meiosis(genome)
    genotype = fertilization(g1,g2)
    genotypes2.append(genotype)
    df_genotypes2 = pd.DataFrame(genotypes2, columns=["Genotype"])

In [ ]:
df_split = df_genotypes["Genotype"].str.split("/", expand=True)
df_split.columns = [f"G{i+1}" for i in range(df_split.shape[1])]

# STEP 2: Encode genotypes as numbers
def encode_genotype(g):
    g = ''.join(sorted(g, key=str.lower))  # Normalize order
    if g[0].isupper() and g[1].isupper():
        return 2  # Homozygous dominant
    elif g[0].islower() and g[1].islower():
        return 0  # Homozygous recessive
    else:
        return 1  # Heterozygous

df_encoded = df_split.applymap(encode_genotype)

# STEP 3: Dimensionality reduction with PCA
pca = PCA(n_components=2)
components = pca.fit_transform(df_encoded)

# STEP 4: K-means clustering
k = 4  # you can change this to any number of clusters
kmeans = KMeans(n_clusters=k, random_state=42)
clusters = kmeans.fit_predict(df_encoded)

# STEP 5: Scatter plot with cluster coloring
plt.figure(figsize=(8, 6))
scatter = plt.scatter(
    components[:, 0],
    components[:, 1],
    c=clusters,
    cmap="tab10",  # or try 'Set2', 'Spectral', 'viridis'
    s=50,
    alpha=0.7
)

plt.title("Genotype Clusters (PCA + K-Means)")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.grid(True)
plt.colorbar(scatter, label="Cluster")
plt.show()

In [21]:
from IPython.display import display, HTML

def calculate_recombination_frequency(data, i, j):
    recombinants = 0
    total = 0

    for genotype in data:
        genes = genotype.split("/")
        g1 = ''.join(sorted(genes[i], key=str.lower))
        g2 = ''.join(sorted(genes[j], key=str.lower))

        het1 = g1[0] != g1[1]
        het2 = g2[0] != g2[1]

        if het1 != het2:  # exactly one is heterozygous
            recombinants += 1

        total += 1

    return recombinants / total if total else 0.0
    
def compute_recombination_matrix(df_genotypes):
    genotypes = df_genotypes["Genotype"].tolist()
    num_loci = len(genotypes[0].split("/"))
    loci_labels = list(string.ascii_uppercase)[:num_loci]

    matrix = pd.DataFrame(index=loci_labels, columns=loci_labels, dtype=float)

    for i in range(num_loci):
        for j in range(num_loci):
            if i == j:
                matrix.iloc[i, j] = 0.0
            elif pd.isna(matrix.iloc[i, j]):
                r = calculate_recombination_frequency(genotypes, i, j)
                matrix.iloc[i, j] = r
                matrix.iloc[j, i] = r  # symmetry

    return matrix

In [23]:
recomb_matrix = 100*compute_recombination_matrix(df_genotypes)
recomb_matrix = recomb_matrix.astype(float)
print(recomb_matrix.values.dtype) 
display(recomb_matrix.round(3))

float64


,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T
A,0.0,0.0,100.0,100.0,100.0,100.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,100.0,0.0
B,0.0,0.0,100.0,100.0,100.0,100.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,100.0,0.0
C,100.0,100.0,0.0,0.0,0.0,0.0,100.0,0.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,0.0,100.0,100.0,0.0,100.0
D,100.0,100.0,0.0,0.0,0.0,0.0,100.0,0.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,0.0,100.0,100.0,0.0,100.0
E,100.0,100.0,0.0,0.0,0.0,0.0,100.0,0.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,0.0,100.0,100.0,0.0,100.0
F,100.0,100.0,0.0,0.0,0.0,0.0,100.0,0.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,0.0,100.0,100.0,0.0,100.0
G,0.0,0.0,100.0,100.0,100.0,100.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,100.0,0.0
H,100.0,100.0,0.0,0.0,0.0,0.0,100.0,0.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,0.0,100.0,100.0,0.0,100.0
I,0.0,0.0,100.0,100.0,100.0,100.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,100.0,0.0
J,0.0,0.0,100.0,100.0,100.0,100.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,100.0,0.0


In [ ]:
def group_loci_by_linkage(recomb_matrix, threshold=46):
    G = nx.Graph()
    loci = list(recomb_matrix.columns)

    # Add all loci as graph nodes
    G.add_nodes_from(loci)

    # Add edges between linked loci (r ≤ threshold)
    for i in range(len(loci)):
        for j in range(i + 1, len(loci)):  # upper triangle only
            locus1 = loci[i]
            locus2 = loci[j]
            r_val = recomb_matrix.loc[locus1, locus2]
            if r_val <= threshold:
                G.add_edge(locus1, locus2)
                print(f"Linking {locus1}-{locus2} with r={r_val:.3f}")

    groups = list(nx.connected_components(G))
    return groups
    
def get_chromosome_matrices(recomb_matrix, groups):
    chrom_matrices = {}
    for idx, group in enumerate(groups, 1):
        group = sorted(group)
        submatrix = recomb_matrix.loc[group, group]
        chrom_matrices[f"Chr{idx}"] = submatrix.round(1)
    return chrom_matrices
    
def display_chromosome_matrices(chrom_matrices):
    html_blocks = []
    for chr_name, matrix in chrom_matrices.items():
        html = f"<h4>{chr_name}</h4>" + matrix.to_html()
        wrapped = f"<div style='display: inline-block; margin-right: 30px; vertical-align: top'>{html}</div>"
        html_blocks.append(wrapped)

    display(HTML(''.join(html_blocks)))

In [ ]:
groups = group_loci_by_linkage(recomb_matrix)
chrom_matrices = get_chromosome_matrices(recomb_matrix, groups)
display_chromosome_matrices(chrom_matrices)


In [ ]:
import numpy as np
from scipy.optimize import basinhopping
from itertools import combinations

def map_genes(chrom_matrix):
    # Extract distances and gene pairs from the matrix
    loci = chrom_matrix.columns.tolist()
    n = len(loci)
    
    # All unique gene pairs
    pairs = list(combinations(range(n), 2))
    
    # Extract target distances from matrix (flattened)
    D = np.array([chrom_matrix.iloc[i, j] for i, j in pairs])
    
    # Objective function: minimize squared error between distances
    def reconstruction_loss(x):
        return sum(abs((abs(x[i] - x[j]) - D[idx])) for idx, (i, j) in enumerate(pairs))

    # Bounds to simulate telomere exclusion
    bounds = [(2, 48)] * n
    x0 = np.random.uniform(2, 48, size=n)

    # Optimization
    minimizer_kwargs = {"method": "L-BFGS-B", "bounds": bounds}
    result = basinhopping(reconstruction_loss, x0, minimizer_kwargs=minimizer_kwargs, niter=20)
    x_recovered = result.x

    # Always keep the unreflected version for output consistency
    print("Mapped positions:")
    for gene, pos in zip(loci, np.round(x_recovered, 2)):
         print(f"{gene}: {pos} cM")
    print("\nTotal absolute error:", round(result.fun, 3))
    
    return x_recovered
    

In [ ]:
#reflect if needed
map_genes(chrom_matrices["Chr1"])
map_genes(chrom_matrices["Chr2"])
map_genes(chrom_matrices["Chr3"])